In [9]:
import json
import urllib2   
query_url = 'http://wservice.viabicing.cat/v2/stations'
response = json.loads(urllib2.urlopen(query_url).read())  


request = "http://wservice.viabicing.cat/v2/stations"

res = json.loads(urllib2.urlopen(request).read())

print(type(res['stations']))

for i in range(0, len(res['stations'])):
	print(res['stations'][i]['id'])
	print('e-bike?\t' + str(res['stations'][i]['type'] == 'BIKE-ELECTRIC'))
	print(res['stations'][i]['latitude'])
	print(res['stations'][i]['longitude'])
	print(res['stations'][i]['slots'])
	print(res['stations'][i]['bikes'])
	print('- ' * 10)

<type 'list'>
1
e-bike?	False
41.397952
2.180042
15
6
- - - - - - - - - - 
2
e-bike?	False
41.39553
2.17706
1
26
- - - - - - - - - - 
3
e-bike?	False
41.393699
2.181137
2
25
- - - - - - - - - - 
4
e-bike?	False
41.39347
2.18149
1
18
- - - - - - - - - - 
5
e-bike?	False
41.391075
2.180223
28
11
- - - - - - - - - - 
6
e-bike?	False
41.391349
2.18061
3
34
- - - - - - - - - - 
7
e-bike?	False
41.388856
2.183251
0
26
- - - - - - - - - - 
8
e-bike?	False
41.389088
2.183568
5
21
- - - - - - - - - - 
9
e-bike?	False
41.385031
2.185249
14
3
- - - - - - - - - - 
10
e-bike?	False
41.38498
2.18417
15
18
- - - - - - - - - - 
11
e-bike?	False
41.381689
2.193914
19
0
- - - - - - - - - - 
12
e-bike?	False
41.384538
2.195679
16
14
- - - - - - - - - - 
13
e-bike?	False
41.386861
2.195761
32
0
- - - - - - - - - - 
14
e-bike?	False
41.384825
2.185074
12
9
- - - - - - - - - - 
15
e-bike?	False
41.39515
2.17076
18
1
- - - - - - - - - - 
16
e-bike?	False
41.39827
2.186708
16
0
- - - - - - - - - - 
17
e-bike?

In [10]:
#create matrix of coordinates of nodes
#d = len(res['stations'])

xy = np.zeros((d,2))

for i in range(0, d):
        x = float((res['stations'][i]['latitude']))
        y = float((res['stations'][i]['longitude']))
        xy[i,:] = x, y
        
#print xy
Xn = xy[:,0]
Yn = xy[:,1]
Xn = Xn.tolist()
Yn = Yn.tolist()

print len(Xn)

465


In [11]:
#set nodes labels to station-ids
labels = []

for i in range(0, d):
        l = (res['stations'][i]['id'])
        labels.append(l)

#print labels
print len(labels)

465


In [12]:
#sara code to get temporal data
import time
import urllib2 
import json
import numpy as np
print "Start : %s" % time.ctime()


def bicing_data():
    t = 0
    all_bicing=[]
    while t < 3:
        print t
        request = 'http://wservice.viabicing.cat/v2/stations'
        res = json.loads(urllib2.urlopen(request).read())
        all_bicing = np.append(all_bicing,res)
        #print res
        t += 1
        time.sleep(5)

    print "End : %s" % time.ctime()
    return all_bicing
    
data = bicing_data()



Start : Tue May 30 18:18:45 2017
0
1
2
End : Tue May 30 18:19:03 2017


In [13]:
# create lists of empty stations and full stations
#for j in range(0,len(data)):

d = len(data[0]['stations'])
empty_stations = []
almost_empty_stations = []
almost_full_stations = []
full_stations = []

for j in range(0,1):
    for i in range(0, d):
        id2 = i
        data[j]['stations'][i].update({'id2':id2})   #add id2 so nodes are labelled in same way as edges
        #calc proportion of bikes in station, and add "ratio" to dict
        if (float((data[j]['stations'][i]['slots']))+float((data[j]['stations'][i]['bikes']))) == 0:
            r = "CLS"
        else: 
            r = float((data[j]['stations'][i]['bikes']))/(float((data[j]['stations'][i]['slots']))+float((data[j]['stations'][i]['bikes'])))
        data[j]['stations'][i].update({'ratio':r})
        #create groupings of stations by proportion full
        if data[j]['stations'][i]['ratio'] < 0.15:
            empty_stations.append(int(data[j]['stations'][i]['id2']))
            data[j]['stations'][i].update({'ratio_label':'empty'})
        if data[j]['stations'][i]['ratio'] >= 0.15 and data[j]['stations'][i]['ratio'] < 0.50:  
            almost_empty_stations.append(int(data[j]['stations'][i]['id2']))
            data[j]['stations'][i].update({'ratio_label':'almost_empty'})
        if data[j]['stations'][i]['ratio'] >= 0.50 and data[j]['stations'][i]['ratio'] < 0.85:
            almost_full_stations.append(int(data[j]['stations'][i]['id2']))
            data[j]['stations'][i].update({'ratio_label':'almost_full'})
        if data[j]['stations'][i]['ratio'] >= 0.85:
            full_stations.append(int(data[j]['stations'][i]['id2']))
            data[j]['stations'][i].update({'ratio_label':'full'}) 



In [14]:
#splitting plotting nodes into 4 groups of empty-to-full
Xn_empty = []
Xn_almost_empty = []
Xn_almost_full = []
Xn_full = []
Yn_empty = []
Yn_almost_empty = []
Yn_almost_full = []
Yn_full = []

for i in empty_stations:
    Xn_empty.append(Xn[i])
    Yn_empty.append(Yn[i])

for i in almost_empty_stations:
    Xn_almost_empty.append(Xn[i])
    Yn_almost_empty.append(Yn[i])
    
for i in almost_full_stations:
    Xn_almost_full.append(Xn[i])
    Yn_almost_full.append(Yn[i])

for i in full_stations:
    Xn_full.append(Xn[i])
    Yn_full.append(Yn[i])
    
#create list of edges
edges = []
for e in empty_stations:
    for f in full_stations:
        x = [e,f]
        edges.append(x)

print len(edges)
#print edges

11310


In [15]:
#plot stations(nodes) as coordinate points
import plotly
from plotly.graph_objs import *
import plotly.plotly as py
plotly.offline.init_notebook_mode(connected=True)

#edges

Xe=[]
Ye=[]
#print edges
for edge in edges:
    Xe+=[Xn[edge[0]],Xn[edge[1]], None]
    Ye+=[Yn[edge[0]],Yn[edge[1]], None]
    
trace1=Scatter(x=Xe,
               y=Ye,
               mode='lines',
               name='edges connecting empty-full',
               line=Line(color='rgb(210,210,210)', width=1),
               hoverinfo='none')
                
trace2=Scatter(x=Xn_empty,
               y=Yn_empty,
               mode='markers',
               name='empty',
               marker=Marker(symbol='dot',
                             size=10, 
                             color= 'red',
                             #colorscale= 'Jet',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

trace3=Scatter(x=Xn_almost_empty,
               y=Yn_almost_empty,
               mode='markers',
               name='almost_empty',
               marker=Marker(symbol='dot',
                             size=10, 
                             color= 'orange',
                             #colorscale= 'Jet',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )
trace4=Scatter(x=Xn_almost_full,
               y=Yn_almost_full,
               mode='markers',
               name='almost_full',
               marker=Marker(symbol='dot',
                             size=10, 
                             color= 'gold',
                             #colorscale= 'Jet',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )
trace5=Scatter(x=Xn_full,
               y=Yn_full,
               mode='markers',
               name='full',
               marker=Marker(symbol='dot',
                             size=10, 
                             color= 'lime',
                             #colorscale= 'Jet',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )


axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

width=800
height=800
layout=Layout(title= "Bicing stations plotted using coordinates",  #include time in title
    font= Font(size=12),
    showlegend=True,
    autosize=False,
    width=width,
    height=height,
    xaxis=XAxis(axis),
    yaxis=YAxis(axis),          
    margin=Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,  
            xref='paper',     
            yref='paper',     
            x=0,  
            y=-0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=Font(
            size=14 
            )     
            )
        ]),           
    )

#this is to add edges
all_data=Data([trace1, trace2, trace3, trace4, trace5])

fig = Figure(data=all_data, layout=layout)

plotly.offline.iplot(fig, filename='BicingStations')

In [172]:
#thinking about distance and weights...
#finding distance between two coordinate points (lat&lon)

import math
import numpy as np

#function to find distance between 2 coordinate points
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = radius * c

    return dist


#create symmetric matrix of distances between 2 nodes
d = len(res['stations'])
A = np.zeros((d,d))

for i in range(0, len(res['stations'])):
    for j in range(i+1, len(res['stations'])):
        #print(res['stations'][i]['id']), (res['stations'][j]['id'])
        lat1 = float((res['stations'][i]['latitude']))
        #print lat1
        long1 = float((res['stations'][i]['longitude']))
        #print long1
        lat2 = float((res['stations'][j]['latitude']))
        #print lat2
        long2 = float((res['stations'][j]['longitude']))
        #print long2
        dist = (distance((lat1, long1), (lat2, long2)) )
        #print dist
        A[i, j] = dist
        A[j, i] = dist
        #print('- ' * 10)
        
print A

[[ 0.          0.36660614  0.4816518  ...,  1.54524915  3.45165     0.87121146]
 [ 0.36660614  0.          0.39637078 ...,  1.18552311  3.08680231
   1.0509322 ]
 [ 0.4816518   0.39637078  0.         ...,  1.39741765  3.22981512
   1.33916288]
 ..., 
 [ 1.54524915  1.18552311  1.39741765 ...,  0.          1.91467454
   1.89592722]
 [ 3.45165     3.08680231  3.22981512 ...,  1.91467454  0.          3.7584962 ]
 [ 0.87121146  1.0509322   1.33916288 ...,  1.89592722  3.7584962   0.        ]]


In [178]:
from copy import copy, deepcopy
import numpy as np

#gaussian function
def gaussian_weights(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

print "Gaussian weights:"
for x in range(13):
    print x, gaussian_weights(x, 0, 4)

#print A[A != 0].min()
#print A[A != 0].max()
A_copy = copy(A)

#Replace elements in distances-matrix with gaussian weights
for i in range(d):
    for j in range(d):
        A_copy[i][j] = gaussian_weights(A_copy[i][j], 0, 4)

print A_copy


[[ 1.          0.99580881  0.99277658 ...,  0.9280974   0.68914127
   0.97656004]
 [ 0.99580881  1.          0.99510235 ...,  0.95702977  0.74247885
   0.9660745 ]
 [ 0.99277658  0.99510235  1.         ...,  0.94080042  0.72181183
   0.94549903]
 ..., 
 [ 0.9280974   0.95702977  0.94080042 ...,  1.          0.8917568
   0.89374983]
 [ 0.68914127  0.74247885  0.72181183 ...,  0.8917568   1.          0.64310538]
 [ 0.97656004  0.9660745   0.94549903 ...,  0.89374983  0.64310538  1.        ]]
